In [39]:
# local python tests
from pprint import pprint
from igraph import *
import igraph.test
import pickle
import os
import numpy as np
from scipy.sparse import csr_matrix
import collections
import copy
from random import randint





"""

    %%%%%%%%%%%%%%%%%%%%%%%%%%% gcn tests %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    x  feature vectors
    tx feature vectors for test (unlabeled)
    allx feature vectors for all

    y  label vectors       numpy.ndarray
    ty test label vectors  numpy.ndarray
    ally  test label of all

    graph : adjacency matrix (dict of lists)
    test.index: node id list for tests?


    steps
    1) inspect ind.citeseer.x/tx/y/ty/allx/ally/test.index

    2) build those files from code

    3) call learning algorithm
        cd src/notebooks
        python gcn_tests_input.py&&  cp ind.* ../gcn/gcn/gcn/data
        cd src/gcn/gcn/gcn
        python train.py --dataset t1


    4) look at embedding result

"""


def pickleToDisk(myobj, filepath):
    pickle.dump(myobj, open(filepath, "wb"))


def printGraphObject(filepath):


    myobject = os.path.basename(filepath)
    print("\n\n*************** "+myobject+" ********************")
    y = pickle.load(open(filepath,'rb'))
    print(type(y))
    if type(y) ==  type([]) or type(y) == type(np.ndarray([])):
        
        print(len(y))
        pprint(y[:min(3,len(y)-1)])
    elif type(y) == type({}) or type(y)==type(collections.defaultdict()):
        pprint(y.keys()[:min(3,len(y)-1)])
        pprint(y[y.keys()[0]])
        pprint(y[y.keys()[1]])
        pprint(y[y.keys()[20]])
        pprint(y[y.keys()[3000]])

    else:
        
        pprint(y.shape)
        pprint(y[0,:min(3,y.shape[1]-1)].toarray())
        pprint(y[0,0].shape)
        pprint(type(y[0,0]))
        pprint(y[0,0])

def readListFile(filepath):
    myobject = os.path.basename(filepath)
    print("\n\n*************** "+myobject+" ********************")
    

    testInstances =[]
    with open(filepath, 'rb') as f:
        for line in f.readlines():
            testInstances.append(int(line))

    pprint(testInstances[:min(3,len(testInstances)-1)])


def inspectData():

    printGraphObject('../gcn/gcn/gcn/data/ind.citeseer.y')
    printGraphObject('../gcn/gcn/gcn/data/ind.citeseer.ty')
    printGraphObject('../gcn/gcn/gcn/data/ind.citeseer.x')
    printGraphObject('../gcn/gcn/gcn/data/ind.citeseer.tx')
    printGraphObject('../gcn/gcn/gcn/data/ind.citeseer.graph')
    readListFile('../gcn/gcn/gcn/data/ind.citeseer.test.index')
    printGraphObject('../gcn/gcn/gcn/data/ind.citeseer.allx')
    printGraphObject('../gcn/gcn/gcn/data/ind.citeseer.ally')


def discretize(elements):
    """ create a binary array for each of the 10 ranges of values between 0 and 1"""
    orig = np.array(elements)
    results = []
    for i in range(10):
        print(i) 
        if i<9:
            range1 = np.bitwise_and(orig >= 0.1*i ,orig <= 0.1*(i+1))
        else:
            range1 = orig >= 0.1*i
        print(range1)
        results.append(range1) 
        
    result = np.transpose(np.asarray(results))
    print(result.shape)
    return result


def generateData(suffix, n_node=1000, n_test=200, n_train=300 ):
    
    

    #igraph.test.run_tests()

    # generation
    #g = Graph.Tree(127, 2)
    #g = Graph.GRG(100, 0.2)
    #g = Graph.Erdos_Renyi(100, 0.2)
    #g = Graph.Watts_Strogatz(1, 100, 4, 0.5 )
    g = Graph.Barabasi(n_node)
    summary(g)

    # graph metrics
    # pprint(g.degree([2,3,6,99]))
    # pprint(g.edge_betweenness())
    # pprint(g.pagerank())
    #pprint(g.get_adjacency())
    #pprint(dir(g))
    


    # test.index
    testList = [] 
    with open(suffix+'.test.index','wb') as f:
        while len(testList) < n_test:
            j = randint(0, n_node-1)
            #verify it is not alerady there
            try:
                pos = testList.index(j)
            except:
                testList.append(j)
            f.write(str.encode(str(j)+"\n"))


    # adjacency dict (graph)
    i = 0
    A = g.get_adjlist()
    graphAdj = {}
    for node in A:
        graphAdj[i] = node
        i+=1
    pickleToDisk(graphAdj, suffix+".graph")

    
    xFeatures = np.ones(n_node)
    # allx is for labeled and unlabeled training samples
    allx  = copy.deepcopy([xFeatures[i] for i in range(n_node) if i not in testList])
    # x is for labeled training samples only
    x = copy.deepcopy(allx[:-500])
    #tx = copy.deepcopy(xFeatures[range(n_node-n_test, n_node)])
    tx = [xFeatures[i] for i in testList]
    

    
    allx = csr_matrix(allx)
    allx = np.transpose(allx)
    pickleToDisk(allx, suffix+".allx")
    x = csr_matrix(x)
    x = np.transpose(x)
    pickleToDisk(x, suffix+".x")
    tx = csr_matrix(tx)
    tx = np.transpose(tx)
    pickleToDisk(tx, suffix+".tx")

    print(x.shape)
    print(tx.shape)
    print(allx.shape)
    


    # labels (ally,y , ty)
    prs = discretize(g.pagerank())

    

    # labels of x, and empty labels of unlabeled training samples
    prs_all = copy.deepcopy([prs[i] for i in range(n_node) if i not in testList])
    # labels of the labeled training samples
    prs_train = copy.deepcopy(prs_all[:-500])
    prs_test = copy.deepcopy([prs[i] for i in testList])
    #testList.extend(validationList)
    #finalList = testList
    #prs_train = [ prs[i] for i in range(prs.shape[0]) if i not in finalList ]

    ally = np.array(prs_all)
    y = np.array(prs_train)  
    ty = np.array(prs_test)


    pickleToDisk(ally, suffix+".ally")
    pickleToDisk(y, suffix+".y")
    pickleToDisk(ty, suffix+".ty")
    
    print(y.shape)
    print(ty.shape)
    print(ally.shape)
    
    

    # run training on those files

In [40]:
generateData(suffix="ind.t3",n_node=4000, n_train=700, n_test=300)
!mv ind.* ./data/

IGRAPH U--- 4000 3999 -- 
(3200, 1)
(300, 1)
(3700, 1)
0
[ True  True  True ...  True  True  True]
1
[False False False ... False False False]
2
[False False False ... False False False]
3
[False False False ... False False False]
4
[False False False ... False False False]
5
[False False False ... False False False]
6
[False False False ... False False False]
7
[False False False ... False False False]
8
[False False False ... False False False]
9
[False False False ... False False False]
(4000, 10)
(3200, 10)
(300, 10)
(3700, 10)


In [50]:
!~/.pyenv/shims/python  train.py --dataset t3


len(y): 3200
labels shape:
(4000, 10)
len(idx_train) 3200
len(idx_val) 500
len(idx_test) 310
features after row normalize and tuple
(array([[   0,    0],
       [   1,    0],
       [   2,    0],
       ...,
       [3997,    0],
       [3998,    0],
       [3999,    0]], dtype=int32),
 array([1., 1., 1., ..., 1., 1., 1.]),
 (4000, 1))
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

2019-01-22 09:43:06.795741: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
weights
dict_keys(['gcn2/graphconvolution_1_vars/weights_0:0', 'gcn2/graphconvolution_2_vars/weights_0:0', 'gcn2/csnembedding_1_vars/weights_0:0', 'gcn2/graphconvolution_3_vars/weights_0:0'])
{'gcn2/csnembedding_1_vars/weights_0:0': array([[-0.48046267, -0.5416126 ],
       [-0.41236466,  0.67351

Epoch: 0004 train_loss= 2.29589 train_acc= 0.04312 val_loss= 2.26873 val_acc= 0.00000 time= 0.01869
weights
dict_keys(['gcn2/graphconvolution_1_vars/weights_0:0', 'gcn2/graphconvolution_2_vars/weights_0:0', 'gcn2/csnembedding_1_vars/weights_0:0', 'gcn2/graphconvolution_3_vars/weights_0:0'])
{'gcn2/csnembedding_1_vars/weights_0:0': array([[-0.44174388, -0.5107383 ],
       [-0.37572375,  0.63534516],
       [-0.37953928,  0.32126525],
       [-0.6289009 , -0.1941057 ],
       [ 0.14025295,  0.009447  ],
       [-0.28694126, -0.2520153 ],
       [-0.30596465,  0.52798814],
       [ 0.24563606,  0.45917714],
       [ 0.45233655,  0.4050945 ],
       [-0.09589331,  0.4228994 ]], dtype=float32),
 'gcn2/graphconvolution_1_vars/weights_0:0': array([[-0.5267645 ,  0.16639407,  0.49219632, -0.39897737, -0.45386815,
        -0.14755037, -0.47153014,  0.29005954, -0.37981144,  0.5344286 ,
        -0.148829  ,  0.14385761,  0.50600845, -0.1242843 , -0.37482354,
         0.3636884 ]], dtype=float32

weights
dict_keys(['gcn2/graphconvolution_1_vars/weights_0:0', 'gcn2/graphconvolution_2_vars/weights_0:0', 'gcn2/csnembedding_1_vars/weights_0:0', 'gcn2/graphconvolution_3_vars/weights_0:0'])
{'gcn2/csnembedding_1_vars/weights_0:0': array([[-0.4129325 , -0.50467294],
       [-0.3576365 ,  0.61018556],
       [-0.37021345,  0.3498805 ],
       [-0.6087965 , -0.2209617 ],
       [ 0.15083373, -0.01412527],
       [-0.3024793 , -0.22808018],
       [-0.30297148,  0.5221785 ],
       [ 0.2455309 ,  0.43462187],
       [ 0.42948008,  0.43376204],
       [-0.10035773,  0.45286763]], dtype=float32),
 'gcn2/graphconvolution_1_vars/weights_0:0': array([[-0.4970274 ,  0.16393714,  0.46862277, -0.36932817, -0.42417493,
        -0.11854647, -0.441825  ,  0.32031634, -0.3501807 ,  0.563669  ,
        -0.11981612,  0.11777989,  0.4868318 , -0.09547634, -0.34519792,
         0.34550637]], dtype=float32),
 'gcn2/graphconvolution_2_vars/weights_0:0': array([[ 0.2784775 ,  0.39531928, -0.11499918,  0.06

weights
dict_keys(['gcn2/graphconvolution_1_vars/weights_0:0', 'gcn2/graphconvolution_2_vars/weights_0:0', 'gcn2/csnembedding_1_vars/weights_0:0', 'gcn2/graphconvolution_3_vars/weights_0:0'])
{'gcn2/csnembedding_1_vars/weights_0:0': array([[-0.39872333, -0.51559997],
       [-0.34966952,  0.5879712 ],
       [-0.34755775,  0.37834743],
       [-0.60163176, -0.24753927],
       [ 0.1507824 , -0.03357524],
       [-0.30913687, -0.20805585],
       [-0.28453228,  0.53285676],
       [ 0.23370588,  0.41330925],
       [ 0.4193893 ,  0.46246678],
       [-0.08273217,  0.48327106]], dtype=float32),
 'gcn2/graphconvolution_1_vars/weights_0:0': array([[-0.46755892,  0.17681254,  0.44922137, -0.34004384, -0.394798  ,
        -0.0906996 , -0.41242313,  0.35141048, -0.32093543,  0.59337884,
        -0.09194834,  0.09383291,  0.4789303 , -0.06809109, -0.3159635 ,
         0.3401893 ]], dtype=float32),
 'gcn2/graphconvolution_2_vars/weights_0:0': array([[ 0.2784775 ,  0.39531928, -0.11499918,  0.06

weights
dict_keys(['gcn2/graphconvolution_1_vars/weights_0:0', 'gcn2/graphconvolution_2_vars/weights_0:0', 'gcn2/csnembedding_1_vars/weights_0:0', 'gcn2/graphconvolution_3_vars/weights_0:0'])
{'gcn2/csnembedding_1_vars/weights_0:0': array([[-0.41008344, -0.53750336],
       [-0.34956217,  0.5686858 ],
       [-0.31949487,  0.40741357],
       [-0.61041135, -0.27513728],
       [ 0.14757209, -0.04828543],
       [-0.31232962, -0.19248036],
       [-0.25832808,  0.5543989 ],
       [ 0.2167753 ,  0.39544144],
       [ 0.42603886,  0.4917819 ],
       [-0.05660826,  0.51456934]], dtype=float32),
 'gcn2/graphconvolution_1_vars/weights_0:0': array([[-0.43846175,  0.19985063,  0.43647933, -0.31126988, -0.3658609 ,
        -0.06458868, -0.38344234,  0.38336268, -0.29223067,  0.62405235,
        -0.06579731,  0.07203544,  0.48424178, -0.0428727 , -0.28727803,
         0.349143  ]], dtype=float32),
 'gcn2/graphconvolution_2_vars/weights_0:0': array([[ 0.2784775 ,  0.39531928, -0.11499918,  0.06

...

{'gcn2/csnembedding_1_vars/weights_0:0': array([[-0.70939857, -0.8187761 ],
       [-0.15990448,  0.59754026],
       [-0.01134342,  0.67607236],
       [-0.9036687 , -0.5544635 ],
       [ 0.42611417,  0.1405654 ],
       [-0.57946974, -0.32518384],
       [ 0.05314225,  0.83525306],
       [ 0.39702475,  0.44916204],
       [ 0.7121544 ,  0.7636655 ],
       [ 0.25262284,  0.7976983 ]], dtype=float32),
 'gcn2/graphconvolution_1_vars/weights_0:0': array([[-0.20893586,  0.4990491 ,  0.6626325 , -0.09831939, -0.14444932,
         0.18898377, -0.1597937 ,  0.68496877, -0.08291644,  0.9158212 ,
         0.19499493,  0.03846435,  0.75215834, -0.04468495, -0.07898141,
         0.6294586 ]], dtype=float32),
 'gcn2/graphconvolution_2_vars/weights_0:0': array([[ 0.2784775 ,  0.39531928, -0.11499918,  0.06734186, -0.07922941,
         0.3507825 ,  0.19360954,  0.36776114, -0.11110106,  0.4120192 ],
       [ 0.280269  , -0.5464994 , -0.72393477,  0.7369411 , -0.7947979 ,
         0.0677678 ,  0.

weights
dict_keys(['gcn2/graphconvolution_1_vars/weights_0:0', 'gcn2/graphconvolution_2_vars/weights_0:0', 'gcn2/csnembedding_1_vars/weights_0:0', 'gcn2/graphconvolution_3_vars/weights_0:0'])
{'gcn2/csnembedding_1_vars/weights_0:0': array([[-0.7366707 , -0.8376862 ],
       [-0.12996535,  0.6080421 ],
       [ 0.01469678,  0.69285786],
       [-0.93115234, -0.5730525 ],
       [ 0.45968464,  0.16405243],
       [-0.61121315, -0.34385613],
       [ 0.08098971,  0.8543178 ],
       [ 0.42773452,  0.46128976],
       [ 0.7379232 ,  0.7803004 ],
       [ 0.2792796 ,  0.81539506]], dtype=float32),
 'gcn2/graphconvolution_1_vars/weights_0:0': array([[-0.1884264 ,  0.5233218 ,  0.68721145, -0.08178014, -0.12589294,
         0.22771458, -0.14070006,  0.7068707 , -0.06723431,  0.9364313 ,
         0.23378275,  0.04703841,  0.7746565 , -0.04658811, -0.06353785,
         0.65281135]], dtype=float32),
 'gcn2/graphconvolution_2_vars/weights_0:0': array([[ 0.2784775 ,  0.39531928, -0.11499918,  0.06

weights
dict_keys(['gcn2/graphconvolution_1_vars/weights_0:0', 'gcn2/graphconvolution_2_vars/weights_0:0', 'gcn2/csnembedding_1_vars/weights_0:0', 'gcn2/graphconvolution_3_vars/weights_0:0'])
{'gcn2/csnembedding_1_vars/weights_0:0': array([[-0.7600014 , -0.85305464],
       [-0.10307398,  0.6170844 ],
       [ 0.03716666,  0.70646995],
       [-0.954911  , -0.58837104],
       [ 0.49008518,  0.18426189],
       [-0.63949555, -0.35971853],
       [ 0.10497972,  0.8697922 ],
       [ 0.4551181 ,  0.47159296],
       [ 0.75990057,  0.7936694 ],
       [ 0.30206293,  0.8296743 ]], dtype=float32),
 'gcn2/graphconvolution_1_vars/weights_0:0': array([[-0.16910088,  0.54385704,  0.7084216 , -0.06687701, -0.10875304,
         0.26460162, -0.1229606 ,  0.72509956, -0.05327368,  0.95345765,
         0.27074602,  0.05510626,  0.7935346 , -0.04792253, -0.04983857,
         0.6725839 ]], dtype=float32),
 'gcn2/graphconvolution_2_vars/weights_0:0': array([[ 0.2784775 ,  0.39531928, -0.11499918,  0.06

weights
dict_keys(['gcn2/graphconvolution_1_vars/weights_0:0', 'gcn2/graphconvolution_2_vars/weights_0:0', 'gcn2/csnembedding_1_vars/weights_0:0', 'gcn2/graphconvolution_3_vars/weights_0:0'])
{'gcn2/csnembedding_1_vars/weights_0:0': array([[-0.7795905 , -0.8654732 ],
       [-0.07968213,  0.6247752 ],
       [ 0.05607687,  0.71746516],
       [-0.97498804, -0.6009037 ],
       [ 0.51660275,  0.20145008],
       [-0.6640548 , -0.37305018],
       [ 0.12529142,  0.88232005],
       [ 0.4789113 ,  0.4802177 ],
       [ 0.7781549 ,  0.80433375],
       [ 0.3211674 ,  0.8411222 ]], dtype=float32),
 'gcn2/graphconvolution_1_vars/weights_0:0': array([[-0.15098003,  0.56102324,  0.7263533 , -0.05360153, -0.09304095,
         0.29836613, -0.10659014,  0.7400889 , -0.04101418,  0.9673602 ,
         0.305019  ,  0.06231277,  0.8090684 , -0.04882186, -0.03785957,
         0.6890505 ]], dtype=float32),
 'gcn2/graphconvolution_2_vars/weights_0:0': array([[ 0.2784775 ,  0.39531928, -0.11499918,  0.06

weights
dict_keys(['gcn2/graphconvolution_1_vars/weights_0:0', 'gcn2/graphconvolution_2_vars/weights_0:0', 'gcn2/csnembedding_1_vars/weights_0:0', 'gcn2/graphconvolution_3_vars/weights_0:0'])
{'gcn2/csnembedding_1_vars/weights_0:0': array([[-0.7958783 , -0.87553483],
       [-0.05955432,  0.6313351 ],
       [ 0.07174207,  0.72631544],
       [-0.9917182 , -0.6110981 ],
       [ 0.53947526,  0.21614724],
       [-0.6850466 , -0.38440633],
       [ 0.1422775 ,  0.8924914 ],
       [ 0.49924943,  0.48750085],
       [ 0.7931569 ,  0.8128776 ],
       [ 0.33710438,  0.8503647 ]], dtype=float32),
 'gcn2/graphconvolution_1_vars/weights_0:0': array([[-0.13407426,  0.57533765,  0.74139863, -0.04192213, -0.07875198,
         0.32964665, -0.09158917,  0.7523389 , -0.03040932,  0.9786816 ,
         0.33669034,  0.06856029,  0.8217676 , -0.04938839, -0.02754978,
         0.7026147 ]], dtype=float32),
 'gcn2/graphconvolution_2_vars/weights_0:0': array([[ 2.78477490e-01,  3.95319283e-01, -1.1499917

In [51]:
!~/.pyenv/shims/python  train.py --dataset cora


len(y): 140
labels shape:
(2708, 7)
len(idx_train) 140
len(idx_val) 500
len(idx_test) 1000
features after row normalize and tuple
(array([[   0, 1274],
       [   0, 1247],
       [   0, 1194],
       ...,
       [2707,  329],
       [2707,  186],
       [2707,   19]], dtype=int32),
 array([0.11111111, 0.11111111, 0.11111111, ..., 0.07692308, 0.07692308,
       0.07692308], dtype=float32),
 (2708, 1433))
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

2019-01-22 09:44:40.859861: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
weights
dict_keys(['gcn2/graphconvolution_1_vars/weights_0:0', 'gcn2/graphconvolution_2_vars/weights_0:0', 'gcn2/csnembedding_1_vars/weights_0:0', 'gcn2/graphconvolution_3_vars/weights_0:0'])
{'gcn2/csnembedding_1_vars/weights

weights
dict_keys(['gcn2/graphconvolution_1_vars/weights_0:0', 'gcn2/graphconvolution_2_vars/weights_0:0', 'gcn2/csnembedding_1_vars/weights_0:0', 'gcn2/graphconvolution_3_vars/weights_0:0'])
{'gcn2/csnembedding_1_vars/weights_0:0': array([[-0.59264356, -0.6625793 ],
       [-0.47720057,  0.79459083],
       [-0.42253146,  0.31833673],
       [-0.8069536 , -0.13958812],
       [ 0.13928159,  0.0377901 ],
       [-0.28366998, -0.37060198],
       [-0.3970146 ,  0.67410237]], dtype=float32),
 'gcn2/graphconvolution_1_vars/weights_0:0': array([[-0.03311247,  0.00589311,  0.02872987, ...,  0.00605156,
        -0.01715421,  0.02185497],
       [ 0.07496443, -0.02762219, -0.02777576, ...,  0.0191644 ,
        -0.02087965,  0.02345341],
       [ 0.05010678,  0.03190571, -0.00761498, ..., -0.00091378,
        -0.00928775, -0.00444426],
       ...,
       [ 0.00305683,  0.00505032,  0.00634061, ..., -0.01442782,
         0.00246597, -0.00797491],
       [ 0.00643744,  0.0448808 ,  0.01370053, .

weights
dict_keys(['gcn2/graphconvolution_1_vars/weights_0:0', 'gcn2/graphconvolution_2_vars/weights_0:0', 'gcn2/csnembedding_1_vars/weights_0:0', 'gcn2/graphconvolution_3_vars/weights_0:0'])
{'gcn2/csnembedding_1_vars/weights_0:0': array([[-0.609721  , -0.6808028 ],
       [-0.49247414,  0.8119704 ],
       [-0.44044125,  0.33473632],
       [-0.8246274 , -0.12132408],
       [ 0.15633467,  0.02015097],
       [-0.30006045, -0.3870002 ],
       [-0.41436622,  0.6900983 ]], dtype=float32),
 'gcn2/graphconvolution_1_vars/weights_0:0': array([[-0.01644155,  0.00170239,  0.01094945, ...,  0.01593417,
         0.00037922,  0.01162509],
       [ 0.08651161, -0.01064625, -0.02558669, ...,  0.02677293,
        -0.00866853,  0.00657483],
       [ 0.05963749,  0.04357709, -0.00637279, ...,  0.00030387,
         0.006131  ,  0.00307939],
       ...,
       [ 0.00651576,  0.01749169,  0.01190513, ...,  0.00332154,
         0.00685834, -0.00464946],
       [ 0.02001846,  0.05160994,  0.02219743, .

{'gcn2/csnembedding_1_vars/weights_0:0': array([[-0.6267012 , -0.69920886],
       [-0.5097197 ,  0.8298968 ],
       [-0.45854068,  0.35269386],
       [-0.8427336 , -0.10386287],
       [ 0.1745993 ,  0.00142579],
       [-0.31716982, -0.40538338],
       [-0.4324536 ,  0.7079312 ]], dtype=float32),
 'gcn2/graphconvolution_1_vars/weights_0:0': array([[-0.00097869,  0.00139842, -0.00325481, ...,  0.00940243,
         0.00963389,  0.01289429],
       [ 0.09407169,  0.00714553, -0.02680097, ...,  0.03860935,
         0.00325816, -0.00845388],
       [ 0.06937379,  0.04174939, -0.0036927 , ...,  0.00519837,
         0.02194297,  0.0072255 ],
       ...,
       [ 0.01551294,  0.03110058,  0.00215846, ...,  0.01729692,
         0.00608564, -0.00733564],
       [ 0.02731025,  0.06183633,  0.01954347, ...,  0.019151  ,
         0.00286623,  0.04662944],
       [ 0.03781337,  0.01340539, -0.02604138, ...,  0.02337163,
        -0.00335054,  0.03962825]], dtype=float32),
 'gcn2/graphconvolution

Epoch: 0009 train_loss= 1.90319 train_acc= 0.26429 val_loss= 1.90475 val_acc= 0.30000 time= 0.02438
weights
dict_keys(['gcn2/graphconvolution_1_vars/weights_0:0', 'gcn2/graphconvolution_2_vars/weights_0:0', 'gcn2/csnembedding_1_vars/weights_0:0', 'gcn2/graphconvolution_3_vars/weights_0:0'])
{'gcn2/csnembedding_1_vars/weights_0:0': array([[-0.65394914, -0.7273634 ],
       [-0.53730226,  0.8579007 ],
       [-0.48758188,  0.38097844],
       [-0.87133896, -0.07573483],
       [ 0.20341502, -0.0279697 ],
       [-0.34428042, -0.434531  ],
       [-0.46098146,  0.7364792 ]], dtype=float32),
 'gcn2/graphconvolution_1_vars/weights_0:0': array([[ 0.00911603, -0.00201563, -0.00256419, ..., -0.00353812,
         0.01883971,  0.02087716],
       [ 0.10647513,  0.02752078, -0.02842921, ...,  0.06253481,
         0.02191504, -0.03464887],
       [ 0.08071927,  0.0374997 ,  0.00638772, ...,  0.01572399,
         0.03936603,  0.01476601],
       ...,
       [ 0.03121206,  0.04324843, -0.01707838, .

...

weights
dict_keys(['gcn2/graphconvolution_1_vars/weights_0:0', 'gcn2/graphconvolution_2_vars/weights_0:0', 'gcn2/csnembedding_1_vars/weights_0:0', 'gcn2/graphconvolution_3_vars/weights_0:0'])
{'gcn2/csnembedding_1_vars/weights_0:0': array([[-1.4345652 , -1.5254194 ],
       [-0.7605059 ,  1.6388924 ],
       [-1.0225699 ,  1.0692829 ],
       [-1.5720627 , -0.06794778],
       [ 0.7861754 , -0.61114955],
       [-1.1700692 , -1.3125173 ],
       [-1.0165898 ,  1.1785161 ]], dtype=float32),
 'gcn2/graphconvolution_1_vars/weights_0:0': array([[ 0.08275053,  0.10565384, -0.09260964, ..., -0.03559096,
        -0.06791246,  0.13029991],
       [ 0.1944073 ,  0.0714988 , -0.12834232, ...,  0.17195636,
         0.06773695, -0.17882219],
       [-0.06908216, -0.09606548,  0.159944  , ..., -0.01981317,
         0.11613172,  0.1263984 ],
       ...,
       [ 0.06821073,  0.04700421, -0.05299825, ...,  0.05653675,
        -0.01958267, -0.03834846],
       [-0.04180601, -0.06262953,  0.18854156, .

Epoch: 0084 train_loss= 0.92550 train_acc= 0.71429 val_loss= 1.13948 val_acc= 0.64200 time= 0.02731
weights
dict_keys(['gcn2/graphconvolution_1_vars/weights_0:0', 'gcn2/graphconvolution_2_vars/weights_0:0', 'gcn2/csnembedding_1_vars/weights_0:0', 'gcn2/graphconvolution_3_vars/weights_0:0'])
{'gcn2/csnembedding_1_vars/weights_0:0': array([[-1.4498286 , -1.5368482 ],
       [-0.7570722 ,  1.6564744 ],
       [-1.02994   ,  1.0852544 ],
       [-1.5886211 , -0.07727018],
       [ 0.7974821 , -0.6222234 ],
       [-1.1857985 , -1.3250657 ],
       [-1.0250262 ,  1.1900043 ]], dtype=float32),
 'gcn2/graphconvolution_1_vars/weights_0:0': array([[ 0.09120653,  0.10930091, -0.09323791, ..., -0.03432672,
        -0.06433346,  0.12588726],
       [ 0.19444323,  0.07336466, -0.12711987, ...,  0.16671823,
         0.06711306, -0.1831235 ],
       [-0.07189538, -0.10478558,  0.16654275, ..., -0.00529126,
         0.12584268,  0.11946101],
       ...,
       [ 0.07889605,  0.04844218, -0.06177042, .

Epoch: 0086 train_loss= 0.96750 train_acc= 0.64286 val_loss= 1.12843 val_acc= 0.63800 time= 0.02551
weights
dict_keys(['gcn2/graphconvolution_1_vars/weights_0:0', 'gcn2/graphconvolution_2_vars/weights_0:0', 'gcn2/csnembedding_1_vars/weights_0:0', 'gcn2/graphconvolution_3_vars/weights_0:0'])
{'gcn2/csnembedding_1_vars/weights_0:0': array([[-1.4669349 , -1.5481663 ],
       [-0.7488634 ,  1.6727315 ],
       [-1.0323715 ,  1.099023  ],
       [-1.6051589 , -0.08767547],
       [ 0.8065493 , -0.6332613 ],
       [-1.203793  , -1.3374437 ],
       [-1.0310919 ,  1.1969686 ]], dtype=float32),
 'gcn2/graphconvolution_1_vars/weights_0:0': array([[ 0.0995474 ,  0.11484855, -0.0941231 , ..., -0.0312186 ,
        -0.05964079,  0.11844004],
       [ 0.19030403,  0.07378529, -0.12359404, ...,  0.162606  ,
         0.07212903, -0.18538253],
       [-0.07193671, -0.1098268 ,  0.16837965, ...,  0.00281604,
         0.1320921 ,  0.11448408],
       ...,
       [ 0.09356368,  0.04972042, -0.07373022, .

Epoch: 0088 train_loss= 0.94086 train_acc= 0.69286 val_loss= 1.12712 val_acc= 0.63800 time= 0.02365
weights
dict_keys(['gcn2/graphconvolution_1_vars/weights_0:0', 'gcn2/graphconvolution_2_vars/weights_0:0', 'gcn2/csnembedding_1_vars/weights_0:0', 'gcn2/graphconvolution_3_vars/weights_0:0'])
{'gcn2/csnembedding_1_vars/weights_0:0': array([[-1.4834287 , -1.5578475 ],
       [-0.73831594,  1.6889794 ],
       [-1.0338974 ,  1.1127639 ],
       [-1.62189   , -0.09562399],
       [ 0.8156405 , -0.6463247 ],
       [-1.2209557 , -1.3494519 ],
       [-1.0361037 ,  1.2051269 ]], dtype=float32),
 'gcn2/graphconvolution_1_vars/weights_0:0': array([[ 0.1064935 ,  0.12032456, -0.09082089, ..., -0.02521247,
        -0.05187041,  0.10810072],
       [ 0.1836789 ,  0.07249425, -0.12145092, ...,  0.16021517,
         0.07961456, -0.18644533],
       [-0.07000007, -0.11255269,  0.16758464, ...,  0.00941741,
         0.13584356,  0.11128167],
       ...,
       [ 0.10107026,  0.04597144, -0.07746443, .

Epoch: 0090 train_loss= 0.91973 train_acc= 0.64286 val_loss= 1.13394 val_acc= 0.63000 time= 0.02533
weights
dict_keys(['gcn2/graphconvolution_1_vars/weights_0:0', 'gcn2/graphconvolution_2_vars/weights_0:0', 'gcn2/csnembedding_1_vars/weights_0:0', 'gcn2/graphconvolution_3_vars/weights_0:0'])
{'gcn2/csnembedding_1_vars/weights_0:0': array([[-1.4970461 , -1.5682532 ],
       [-0.73381585,  1.7056653 ],
       [-1.0383997 ,  1.1280981 ],
       [-1.6366513 , -0.09925158],
       [ 0.8249141 , -0.66220504],
       [-1.234148  , -1.3613572 ],
       [-1.041982  ,  1.2162186 ]], dtype=float32),
 'gcn2/graphconvolution_1_vars/weights_0:0': array([[ 0.11232092,  0.12306517, -0.08432063, ..., -0.02019557,
        -0.04400083,  0.10209687],
       [ 0.17499192,  0.06898217, -0.11967734, ...,  0.16144817,
         0.09010113, -0.18846315],
       [-0.07038743, -0.11350553,  0.166863  , ...,  0.01186716,
         0.13527901,  0.11112431],
       ...,
       [ 0.1035748 ,  0.03947915, -0.07751888, .